In [2]:
from copy import deepcopy
import gurobipy as gp
from gurobipy import GRB
import torch
import torch.nn as nn
import numpy as np
import os
import math
import time
from cutting_plane_framework import CuttingPlaneMethod
import sys
sys.path.append('./code_LY')
from model import var_sorter
from tqdm import tqdm


In [8]:

class NaiveCuttingPlaneTreeAlgorithm(CuttingPlaneMethod):
    def __init__(self, instanceName, maxIteration=100, 
                    OutputFlag=0, Threads=1, MIPGap=0.0, TimeLimit=3600, MIPFocus=2, cglp_OutputFlag=0, 
                        cglp_Threads=1, cglp_MIPGap=0.0, cglp_TimeLimit=100, cglp_MIPFocus=0, 
                            addCutToMIP=False, number_branch_var=2, normalization='SNC', 
                                training=True, load_ckp=True, save_interval=20
                            ):
        super().__init__(instanceName, maxIteration, OutputFlag, Threads, MIPGap, TimeLimit, MIPFocus, cglp_OutputFlag, cglp_Threads, cglp_MIPGap, cglp_TimeLimit, cglp_MIPFocus, addCutToMIP, number_branch_var, normalization)
        self.tensorA = None
        self.col_feature = None
        self.row_feature = None
        # model
        self.pred=var_sorter(v_size=6,c_size=3,sample_sizes=[64],multi_head=2,natt=2)
        # optimizer
        self.optimizer = torch.optim.Adam(self.pred.parameters(),lr=1e-4)
        # training params
        self.training = training
        self.lastLogit = None
        self.save_interval = save_interval
        if not os.path.isdir('./models'): # create model folder if there is no models folder
            os.mkdir('./models')
        self.ckp_starter = 0
        if load_ckp: # checkpoint: load model and optimizer; start from the latest checkpoint
            '''get latest checkpoint'''
            #  resume model training from the latest checkpoint, allowing the training to continue from the state it was in when interrupted, rather than starting over from the beginning.
            ckps = os.listdir('./models') # get all ckps (files with .mdl extension) in this directory
            if len(ckps) > 0:
                ckps.sort(key=lambda x:int(x.replace('.mdl','').split('_')[-1]))
                tar_name = f'./models/{ckps[-1]}'
                # load
                cpu_dev = torch.device('cpu')
                checkpoint = torch.load(tar_name,map_location=cpu_dev)  # load model and optimizer
                self.pred.load_state_dict(checkpoint['model'])          # load model
                self.ckp_starter = checkpoint['nepoch'] 
                print(f'Loaded check point: {tar_name}')
            else:
                print('No check point to load')
        #set up log files
        if not os.path.isdir('./logs'):
            os.mkdir('./logs')
        self.log_file = open('./logs/training.log','w') # 'w' means: if the file exists, it will be overwritten
            
    
    def A_to_sparse_tensor(self):
        self.tensorA = self.A.tocoo()
        indices = np.vstack((self.tensorA.row,self.tensorA.col))
        data = self.tensorA.data
        self.tensorA = torch.sparse_coo_tensor(indices, data, self.tensorA.shape, dtype=torch.float32)
        
    def feat_extract(self):
        #A matrix update
        self.A_to_sparse_tensor()

        #variable feature update
        if self.col_feature is None:
            #first time constructing 
            self.col_feature = [None] * len(self.variables)
            for var in self.variables:   
                rel_sol_indx= self.varName_map_position[var.VarName]
                col_value = self.lp_sol[rel_sol_indx]
                # if abs(col_val-round(col_val,0))>1e-6 and (vtp=='B' or vtp=='I'):
                #     cand.append(col_var.VarName)
                
                #compose features
                isBin=0
                isInt=0
                if var.VType == 'I':
                    isInt=1
                elif var.VType == 'B':
                    isBin=1
                # ---- [lp_sol, LB, UB, isBin, isInt, reduced_cost, lp_obj]
                self.col_feature[rel_sol_indx] = [col_value, self.LB[rel_sol_indx], self.UB[rel_sol_indx], isBin, isInt, var.Obj]  
            self.col_feature=torch.as_tensor(self.col_feature, dtype=torch.float32)  
        else:
            # not first time, update bound and col_val
            for var in self.variables:   
                rel_sol_indx = self.varName_map_position[var.VarName]
                col_val = self.lp_sol[rel_sol_indx]
                self.col_feature[rel_sol_indx][0] = col_val
                
        #constraint feature update
        row_index_map={}
        rcounter=0
        row_feat=[]
        for idx, constr in enumerate(self.lp_relaxation.getConstrs()):
            if constr.ConstrName not in row_index_map:
                row_index_map[constr.ConstrName] = rcounter
                rcounter += 1
            #compose row feature
            sense1 = 0
            sense2 = 0 # 01:leq 00:eq 10:geq
            if constr.Sense == '<':
                sense2 = 1
            elif constr.Sense == '>':
                sense1 = 1 
            row_feat.append([constr.RHS, sense1, sense2]) # [RHS, SENSE]
        row_feat = torch.as_tensor(row_feat, dtype=torch.float32)
        cand = torch.as_tensor([self.varName_map_position[x] for x in self.Cand])
        return  row_feat, row_index_map, cand
        
    
    def variable_selection(self, ifTrain = True, variable_selection_way = 'MFR'):
        # choose the variable to branch
        #TODO: add criteria to select either explore or exploit
        # when we will do heuristic branching, when we will explore, and when we will exploit?
        if variable_selection_way == 'RL':
            # the number of variables that are chosen to branch, so the number of nodes in the branching tree is 2^number_of_candidates            
            number_of_candidates = self.number_branch_var  
            # get feature, candidates are stored in self.Cand
            row_feat, row_index_map, cand = self.feat_extract()
            
            # calling model to do prediction
            if ifTrain:
                self.optimizer.zero_grad()
            self.lastLogit = self.pred(self.tensorA, self.col_feature, row_feat)
            decision = torch.argsort(torch.index_select(self.lastLogit, 0, cand),descending=True)[:number_of_candidates]
            self.branchVar[self.iteration-1] = {}
            for i in decision:
                tar_var = self.Cand[i]
                # ori_ind = cand[i].item()
                self.branchVar[self.iteration-1][tar_var] = self.lastLogit[i] # self.lp_sol[ori_ind]
        elif variable_selection_way == 'MFR':
            # heuristic: Maximum Fractionality Rule
            number_of_candidates = self.number_branch_var                                                                       # the number of variables that are chosen to branch, so the number of nodes in the branching tree is 2^number_of_candidates
            number_of_noninteger = len(self.non_integer_vars[self.iteration - 1])
            number_of_nonbinary = len(self.non_binary_vars[self.iteration - 1])
            self.branchVar[self.iteration-1] = {}
            if number_of_noninteger > 0:
                list1 = sorted(self.non_integer_vars[self.iteration-1].items(), key=lambda x: x[1], reverse=True)[:number_of_candidates]   # find the integer variables that have the largest distance to the nearest integer
                if len(list1) <= number_of_candidates:
                    for item in list1:
                        self.branchVar[self.iteration-1][item[0]] = item[1] # 
                else:
                    for item in list1[0:number_of_candidates]:
                        self.branchVar[self.iteration-1][item[0]] = item[1]

            if number_of_nonbinary > 0: 
                list2 = sorted(self.non_binary_vars[self.iteration-1].items(), key=lambda x: x[1], reverse=True)[:number_of_candidates]    # find the binary variables that have the largest distance to {0,1}
                if len(list2) <= number_of_candidates:
                    for item in list2:
                        self.branchVar[self.iteration-1][item[0]] = item[1]
                else:
                    for item in list2[0:number_of_candidates]:
                        self.branchVar[self.iteration-1][item[0]] = item[1]
        


    def branching_tree_building(self, node, level, varInfo):
        if level == len(self.branchVar[self.iteration-1]):
            return
        else:
            varName, info = list(varInfo.items())[level]
            pos = self.varName_map_position[varName]

            left_node = {}
            left_node['LB'] = deepcopy(self.nodeSet[node]['LB'])
            left_node['LB'][pos] = info['upper']
            left_node['UB'] = deepcopy(self.nodeSet[node]['UB'])
            left_node['trace'] = deepcopy(self.nodeSet[node]['trace'])
            left_node['trace'].append('l')
            
            right_node = {}
            right_node['LB'] = deepcopy(self.nodeSet[node]['LB'])
            right_node['UB'] = deepcopy(self.nodeSet[node]['UB'])
            right_node['UB'][pos] = info['lower']
            right_node['trace'] = deepcopy(self.nodeSet[node]['trace'])
            right_node['trace'].append('r')

            left_node_ind = max(self.nodeSet.keys()) + 1
            right_node_ind = left_node_ind + 1
            self.nodeSet[left_node_ind] = left_node
            self.nodeSet[right_node_ind] = right_node
            del self.nodeSet[node]  

            self.branching_tree_building(left_node_ind, level+1, varInfo)
            self.branching_tree_building(right_node_ind, level+1, varInfo)
     
    def branching_tree(self):
        varInfo = {}
        for varName in self.branchVar[self.iteration-1].keys():
            varInfo[varName] = {}
            varInfo[varName]['val'] = self.lp_relaxation.getVarByName(varName).x
            varInfo[varName]['lower'] = math.floor(varInfo[varName]['val'])
            varInfo[varName]['upper'] = math.ceil(varInfo[varName]['val'])
        
        self.nodeSet = {}
        self.nodeSet[0] = {}
        self.nodeSet[0]['LB'] = deepcopy(self.LB)
        self.nodeSet[0]['UB'] = deepcopy(self.UB)
        self.nodeSet[0]['trace'] = []
        self.branching_tree_building(0, 0, varInfo) 
    
    def solve(self, ifTrain = True, variable_selection_way = 'RL'):
        time_init = time.time()
        while self.iteration <= self.maxIteration:
            iter_begin = time.time()
            self.master_problem()
            if self.OPT == True:
                self.print_iteration_info()
                return
            self.variable_selection(ifTrain, variable_selection_way)
            self.branching_tree()
            ready_to_cut = time.time()
            self.cut_generation()
            iter_end = time.time()
            overall = iter_end - time_init
            iteration_time = iter_end - iter_begin
            cut_time = iter_end - ready_to_cut
            self.print_iteration_info(cut_time, iteration_time, overall)


    def train_model_each_iteration(self):
        time_init = time.time()
        while self.iteration <= self.maxIteration:
            iter_begin = time.time()
            self.master_problem()
            if self.OPT == True:
                self.print_iteration_info()
                return
            self.variable_selection(True, 'RL')
            self.branching_tree()
            ready_to_cut = time.time()
            self.cut_generation()
            iter_end = time.time()
            overall = iter_end - time_init
            iteration_time = iter_end - iter_begin
            cut_time = iter_end - ready_to_cut
            self.print_iteration_info(cut_time, iteration_time, overall)
            if self.iteration > 1 and self.training:
                # compute improvement
                improvement = (self.lp_obj_value[self.iteration - 1] - self.lp_obj_value[self.iteration - 2]) / self.lp_obj_value[0] * 100
                # record reward
                self.log_file.write(f'iter:{self.iteration} reward:{improvement}\n')
                # update gradient and update model
                regret = torch.sum(self.lastLogit * ((-1.0) * improvement + 1e-6))
                regret.backward()
                self.optimizer.step()
                # check if need to save model
                if (self.iteration-1) % self.save_interval == 0 and self.iteration-1 != 0:
                    state = {'model':self.pred.state_dict(),'optimizer':self.optimizer.state_dict(),'nepoch':self.ckp_starter+self.iteration-1}
                    torch.save(state,f'./models/ckp_{self.iteration+self.ckp_starter-1}.mdl')
    
    def train_model_each_round(self, num_episodes = 500):
        #TODO: train model after each round, not each iteration; and for each round, we need to initialize the model (cuts, variables, etc.):
        '''         # Initialize the model:
                    self.iteration = 0
                    self.lp_relaxation = self.mipModel.relax()
                    self.lp_relaxation.update()
                    self.coefList = {}
                    self.lp_obj_value = {}
        '''
        for i in range(10):
            with tqdm(total=int(num_episodes / 10), desc='Iteration %d' % i) as pbar:
                for i_episode in range(int(num_episodes / 10)):
                    time_init = time.time()
                    self.iteration = 0
                    self.lp_relaxation = self.mipModel.relax()
                    self.lp_relaxation.update()
                    self.coefList = {}
                    self.lp_obj_value = {}
                    while self.iteration <= self.maxIteration:
                        iter_begin = time.time()
                        self.master_problem()
                        self.variable_selection()
                        self.branching_tree()
                        ready_to_cut = time.time()
                        self.cut_generation()
                        iter_end = time.time()
                        overall = iter_end - time_init
                        iteration_time = iter_end - iter_begin
                        cut_time = iter_end - ready_to_cut
                        self.print_iteration_info(cut_time, iteration_time, overall)
                        
                    # compute improvement
                    improvement = abs(self.lp_obj_value[self.iteration - 1] - self.lp_obj_value[0]) / self.lp_obj_value[0] * 100
                    # record reward
                    self.log_file.write(f'episode:{i * 10 + i_episode + 1} reward:{improvement}\n')
                    # update gradient and update model
                    regret = torch.tensor(-improvement, requires_grad=True)
                    regret.backward()
                    self.optimizer.step()
                    # check if need to save model
                    # if (self.iteration-1) % self.save_interval == 0 and self.iteration-1 != 0:
                    #     state = {'model':self.pred.state_dict(),'optimizer':self.optimizer.state_dict(),'nepoch':self.ckp_starter+self.iteration-1}
                    #     torch.save(state,f'./models/ckp_{self.iteration+self.ckp_starter-1}.mdl')
                    # pbar.update(1)        
     

In [10]:
instanceName = '50v-10'
cpt = NaiveCuttingPlaneTreeAlgorithm(instanceName,training=True, save_interval=20, maxIteration=100, OutputFlag = 0, Threads = 1, MIPGap = 0.0, TimeLimit = 300, number_branch_var = 2, normalization = 'SNC')
cpt.solve(variable_selection_way = 'MFR')

Read MPS format model from file benchmark/50v-10.mps.gz
Reading time = 0.03 seconds
50v-10: 233 rows, 2013 columns, 2745 nonzeros
Loaded check point: ./models/ckp_500.mdl
This problem has 1647 integer variables and 0 binary variables.
The optimal value of LP relaxation is 2879.0656868536717.
---------------------------------------------------------------------------------------------------------------------------------
|  Iter  |  # fractional var  |  current value  |  Relative Improvement  |  Overall Improvement  |  Iter Time  |  Overall Time  |
---------------------------------------------------------------------------------------------------------------------------------
|       1|                  30|        2882.7344|                 0.1273 |                0.1274 |      1.1397 |         1.8842 |
|       2|                  30|        2885.5635|                 0.0980 |                0.2257 |      1.2072 |         3.0915 |
|       3|                  29|        2888.0303|        

In [11]:
instanceName = '50v-10'
cpt = NaiveCuttingPlaneTreeAlgorithm(instanceName,training=True, save_interval=20, maxIteration=100, OutputFlag = 0, Threads = 1, MIPGap = 0.0, TimeLimit = 300, number_branch_var = 2, normalization = 'SNC')
cpt.train_model_each_iteration()

Read MPS format model from file benchmark/50v-10.mps.gz
Reading time = 0.03 seconds
50v-10: 233 rows, 2013 columns, 2745 nonzeros
Loaded check point: ./models/ckp_500.mdl
This problem has 1647 integer variables and 0 binary variables.
The optimal value of LP relaxation is 2879.0656868536717.
---------------------------------------------------------------------------------------------------------------------------------
|  Iter  |  # fractional var  |  current value  |  Relative Improvement  |  Overall Improvement  |  Iter Time  |  Overall Time  |
---------------------------------------------------------------------------------------------------------------------------------
|       1|                  29|        2887.4657|                 0.2909 |                0.2918 |      1.1777 |         2.0126 |
|       2|                  29|        2887.4657|                 0.0000 |                0.2918 |      1.1169 |         3.1966 |
|       3|                  29|        2892.4657|        